# Eagle 1T - 2nd stage calibration 

Various smaller runs to figure out the optimal settings for the model calibration

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="C2-7B2T"
DEEPSPEED_STRAT="deepspeed_stage_2"

EXPERIMENT_NAME="enwiki-16k-5e-5-no-cuda"
LEARNING_RATE="5e-5"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# The model sizing
MODEL_PATH="/workspace/main-models/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /workspace/picocreator/RWKV-infctx-trainer/notebook/major-runs/Eagle-2T-calibration
TRAINER_DIR: /workspace/picocreator/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /workspace/picocreator/RWKV-infctx-trainer


In [2]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/enwiki-16k-rechunk.yaml"

^C
Traceback (most recent call last):
  File "/workspace/picocreator/RWKV-infctx-trainer/RWKV-v5/preload_datapath.py", line 5, in <module>
    from src.data import RWKVDataModule
  File "/workspace/picocreator/RWKV-infctx-trainer/RWKV-v5/src/data.py", line 1, in <module>
    from lightning import LightningDataModule
  File "/root/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/__init__.py", line 18, in <module>
    from lightning.fabric.fabric import Fabric  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/__init__.py", line 29, in <module>
    from lightning.fabric.fabric import Fabric  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/fabric.py", line 35, in <module>
    import torch
  File "/root/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/torch/__init__.py"

# Enwiki 16 - 3e-5 with WITH CUDA

In [ ]:
!cd "{TRAINER_DIR}" && \
    export RWKV_NO_CUDA=1 && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/enwiki-16k-rechunk.yaml" \
        --model.load_model="{MODEL_PATH}" \
        --model.lr_init={LEARNING_RATE} \
        --model.lr_final={LEARNING_RATE} \
        --data.skip_datapath_setup=True \
        --trainer.callbacks.init_args.dirpath="/checkpoint/calibration/Eagle-2T/{EXPERIMENT_NAME}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - {EXPERIMENT_NAME} ({DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.target_batch_size=1024 \
        --trainer.microbatch_size=8 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-04 04:35:14,774] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/root/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/workspace/picocreator/RWKV-infctx-trainer/notebook/major-runs/Eagle-2T-calibration/enwiki-16k-rechunk.yaml', '--model.load_model=/workspace/main-models/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', '--model.lr_init=3e-5', '--model.lr_final=3e-5', '--data.skip_datapath_setup=True', '--trainer.callbacks.init_args.dirpath=/checkpoint/calibration/Eagle-2T/enwiki-16k-3e-5-with-cuda/', '--trainer.logger.init_args.name=C2-7B2T - enwiki-16k-3e-5-with-cuda (deepspeed_stage_2)', '--train